# Cluster Plot

In [ ]:
import geopandas as gpd
import pandas as pd
import json
import random
from jupyter_dash import JupyterDash
from dash import html
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.javascript import assign

In [ ]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

### Prepare the Data
Note I added a tooltip. It works the same way as boundaries

In [ ]:
df = pd.read_csv('data/uscities.csv')  # https://simplemaps.com/data/us-cities
cities = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat))

statecities = cities[cities.state_id == 'NJ'].copy(deep=True)
statecities['tooltip'] = statecities.city

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

#### Convert the data to geobuf then setup color bars as before

In [ ]:
geojson = json.loads(statecities.to_json())
geobuf = dlx.geojson_to_geobuf(geojson)  # convert to geobuf

In [ ]:
colorscale = ['red', 'yellow', 'green', 'blue', 'purple']  # rainbow
chroma = "https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"  # js lib used for colors
color_prop = 'density'

vmax = df[color_prop].max()
colorbar = dl.Colorbar(colorscale=colorscale,
                       width=20,
                       height=150,
                       min=0,
                       max=vmax,
                       unit='/km2')
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
point_to_layer = assign("""function(feature, latlng, context){
    const {min, max, colorscale, circleOptions, colorProp} = context.props.hideout;
    const csc = chroma.scale(colorscale).domain([min, max]);  // chroma lib to construct colorscale
    circleOptions.fillColor = csc(feature.properties[colorProp]);  // set color based on color prop.
    return L.circleMarker(latlng, circleOptions);  // sender a simple circle marker.
}""")

## Clustering
Based on mapbox's supercluster: https://github.com/mapbox/supercluster. You can pass cluster options to supercluster using the `superClusterOptions` property. 
### Key superClusterOptions
* `minZoom`	- Minimum zoom level at which clusters are generated.
* `maxZoom`	- Maximum zoom level at which clusters are generated.
* `minPoints`	-	Minimum number of points to form a cluster.
* `radius` - Cluster radius.	

### zoomToBoundsOnClick behavior
If turned on clicking on a cluster zooms in until based on clustering rules, the cluster subdivides.

In [ ]:
cities = dl.GeoJSON(data=geobuf,
                    id="geojson",
                    format="geobuf",
                    options=dict(pointToLayer=point_to_layer),
                    cluster=False,
                    zoomToBoundsOnClick=True,
                    superClusterOptions=dict(radius=40),
                    hideout=dict(colorProp=color_prop,
                                 circleOptions=dict(fillOpacity=1,
                                                    stroke=False,
                                                    radius=3),
                                 min=0,
                                 max=vmax,
                                 colorscale=colorscale))

app = JupyterDash(external_scripts=[chroma])
app.layout = html.Div([
    dl.Map(children=[dl.TileLayer(), cities, colorbar],
           center=[40.5, -73],
           zoom=7,
           style={
               'width': '1000px',
               'height': '500px'
           },
           id="map")
])
app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

### Reminder most component properties can be changed in a callback
For fun the `cluster` property is toggled here

In [ ]:
from dash.dependencies import Input, Output
cities = dl.GeoJSON(data=geobuf,
                    id="geojson",
                    format="geobuf",
                    options=dict(pointToLayer=point_to_layer),
                    cluster=True,
                    superClusterOptions=dict(radius=100, minPoints=5),
                    hideout=dict(colorProp=color_prop,
                                 circleOptions=dict(fillOpacity=1,
                                                    stroke=False,
                                                    radius=3),
                                 min=0,
                                 max=vmax,
                                 colorscale=colorscale))

app = JupyterDash(external_scripts=[chroma])
app.layout = html.Div([
    dl.Map(children=[dl.TileLayer(), cities, colorbar],
           center=[40.5, -73],
           zoom=7,
           style={
               'width': '1000px',
               'height': '500px'
           },
           id="map"),
    html.Button("Click Me!", id='btn')
])


@app.callback(Output('geojson', 'cluster'), Input('btn', 'n_clicks'))
def filter(input_):
    if input_:
        return (input_ % 2 == 1)


app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

In [ ]:
# Changing the type of marker
cluster_to_layer1 = assign("""function(feature, latlng, index, context){
    
    // Render a circle with the number of leaves written in the center.
    const icon = L.divIcon.scatter({
        html: '<div style="background-color:white;"><span>' + feature.properties.point_count_abbreviated + '</span></div>',
        className: "marker-cluster",
        iconSize: L.point(40, 40),
        color: 'black'
    });
    return L.marker(latlng, {icon : icon})
}""")

In [ ]:
# Matching colorbar
cluster_to_layer2 = assign("""function(feature, latlng, index, context){
    const {min, max, colorscale, circleOptions, colorProp} = context.props.hideout;
    const csc = chroma.scale(colorscale).domain([min, max]);
    // Set color based on mean value of leaves.
    const leaves = index.getLeaves(feature.properties.cluster_id);
    let valueSum = 0;
    for (let i = 0; i < leaves.length; ++i) {
        valueSum += leaves[i].properties[colorProp]
    }
    const valueMean = valueSum / leaves.length;
    // Render a circle with the number of leaves written in the center.
    const icon = L.divIcon.scatter({
        html: '<div style="background-color:white;"><span>' + feature.properties.point_count_abbreviated + '</span></div>',
        className: "marker-cluster",
        iconSize: L.point(40, 40),
        color: csc(valueMean)
    });
    return L.marker(latlng, {icon : icon})
}""")

In [ ]:
# Adding tooltip
cluster_to_layer3 = assign("""function(feature, latlng, index, context){
    const {min, max, colorscale, circleOptions, colorProp} = context.props.hideout;
    const csc = chroma.scale(colorscale).domain([min, max]);
    // Set color based on mean value of leaves.
    const leaves = index.getLeaves(feature.properties.cluster_id);
    let valueSum = 0;
    for (let i = 0; i < leaves.length; ++i) {
        valueSum += leaves[i].properties[colorProp]
    }
    const valueMean = valueSum / leaves.length;
    // Render a circle with the number of leaves written in the center.
    feature.properties.tooltip='Number: '+feature.properties.point_count_abbreviated+'<BR> Mean Value: '+valueMean;
    const icon = L.divIcon.scatter({
        html: '<div style="background-color:white;"><span>' + feature.properties.point_count_abbreviated + '</span></div>',
        className: "marker-cluster",
        iconSize: L.point(40, 40),
        color: csc(valueMean)
    });
    return L.marker(latlng, {icon : icon})
}""")

In [ ]:
cities = dl.GeoJSON(
    data=geobuf,
    id="geojson",
    format="geobuf",
    cluster=True,
    clusterToLayer=cluster_to_layer3,
    zoomToBoundsOnClick=True,
    options=dict(pointToLayer=point_to_layer),
    superClusterOptions=dict(radius=150),  # adjust cluster size
    hideout=dict(colorProp=color_prop,
                 circleOptions=dict(fillOpacity=1, stroke=False, radius=5),
                 min=0,
                 max=vmax,
                 colorscale=colorscale))

app = JupyterDash(external_scripts=[chroma])
app.layout = html.Div([
    dl.Map(children=[dl.TileLayer(), cities, colorbar],
           center=[40.5, -73],
           zoom=7,
           style={
               'width': '1000px',
               'height': '500px'
           },
           id="map")
])
app.run_server(mode='inline', port=random.choice(range(2000, 10000)))